In [2]:
import sys
sys.path.insert(1, '../') 
from utils.loader import Loader
import tensorflow as tf
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten, Input, Concatenate, BatchNormalization, LSTM
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

# Descargar la lista de stopwords si no está ya descargada
nltk.download('stopwords')

# Cargar las stopwords en español
spanish_stopwords = set(stopwords.words('spanish'))


print("¿GPU disponible?:", tf.config.list_physical_devices('GPU'))

¿GPU disponible?: []


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lingf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
train = Loader.load_NLP()[:20000]

c:\Users\lingf\OneDrive\Escritorio\SRC\Cars_Price_Prediction-1\NLP\..\utils\loader.py:153: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path,
c:\Users\lingf\OneDrive\Escritorio\SRC\Cars_Price_Prediction-1\NLP\..\utils\loader.py:153: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path,
c:\Users\lingf\OneDrive\Escritorio\SRC\Cars_Price_Prediction-1\NLP\..\utils\loader.py:165: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform

In [4]:
descriptions = [col for col in train.columns if "description" in col]

print(descriptions)

def custom_concat(row, cols):
    # Construir la descripción con lógica condicional basada en el valor de la celda
    parts = []
    for col_name in cols:  # Cambio para iterar solo sobre las columnas especificadas
        if col_name in row.index:  # Verificar que el nombre de la columna esté en el DataFrame
            value = row[col_name]
            if value == "no tiene" or not isinstance(value, str):
                parts.append(f"no tiene {col_name}")
            else:
                parts.append(str(value))  # Convertir a string para evitar problemas al unir
    # Unir todas las partes con espacios
    return ' '.join(parts)

# Aplicar la función al DataFrame
train['full_description'] = train.apply(custom_concat, axis=1, args=(descriptions,))
#train[descriptions].apply(lambda col: col.apply(lambda x: type(x).__name__).value_counts())

#descriptions.append("idx")
#descriptions.append("price")
#descriptions.append("km")

['motor_description', 'transmission_description', 'parking_sensors_description', 'Exterior_Llantas_description', 'Exterior_Molduras_description', 'Exterior_Neumáticos_description', 'Exterior_Puerta_description', 'Exterior_Retrovisor_description', 'Confort_Volante_description', 'seat_description', 'head_supporter_description']


In [5]:
filtered_columns = ["price", "km", "fuelType", "full_description"]
train = train[filtered_columns]
train.dropna(inplace=True)
train

,price,km,fuelType,full_description
idx,,,,
0,10600,99700.0,Gasolina,"Motor de 1,8 litros ( 1.798 cc ) , 4 cilindros..."
1,12450,91924.0,Gasolina,"Motor de 1,4 litros ( 1.362 cc ) , cuatro cili..."
2,134900,62000.0,Gasolina,"Motor de 3,8 litros ( 3.800 cc ) , seis cilind..."
3,27500,13500.0,Eléctrico,no tiene motor_description Transmisión de tipo...
4,27990,13559.0,Híbrido,"Motor de 1,5 litros ( 1.498 cc ) , cuatro cili..."
...,...,...,...,...
20029,7500,74000.0,Gasolina,"Motor de 1,2 litros ( 1.198 cc ) , tres cilind..."
20030,21990,10.0,Híbrido,"Motor de 1,0 litros ( 998 cc ) , tres cilindro..."
20031,22990,148000.0,Diésel,"Motor de 2,0 litros ( 1.968 cc ) , cuatro cili..."


In [6]:
#Parameters
verb_size = 150

In [7]:
def preprocess_text(text):
    # Convertir a minúsculas y eliminar caracteres especiales (pero mantener los números)
    text = text.lower()
    text = re.sub(r'[^\w\s\d]', '', text)  # Eliminar signos de puntuación, pero mantener números
    tokens = text.split()
    
    # Filtrar tokens, eliminando las stopwords
    filtered_tokens = [token for token in tokens if token not in spanish_stopwords]
    
    return filtered_tokens

train['tokens'] = train['full_description'].apply(preprocess_text)
model_w2v = Word2Vec(sentences=train['tokens'], vector_size=verb_size, window=1, min_count=3, workers=8)
word_vectors = model_w2v.wv

def get_average_embedding(tokens, model):
    embeddings = [model[word] for word in tokens if word in model]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.vector_size) 
train['embedding'] = train['tokens'].apply(lambda x: get_average_embedding(x, word_vectors))

In [12]:
x_km = train['km'].to_numpy().reshape(-1, 1)
scaler_km = StandardScaler()
x_km_scaled = scaler_km.fit_transform(x_km)

y = train['price'].to_numpy()
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

In [13]:
# Load data (assuming df is your DataFrame containing the required columns)
texts = train['full_description'].values  # descriptions
embeddings = train['embedding'].values  # embeddings

# Tokenization and sequence padding
tokenizer = Tokenizer(num_words=verb_size)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
data = pad_sequences(sequences,  maxlen=verb_size, padding='post')

In [14]:
# Split data into training and testing sets
train_data, test_data, train_embeddings, test_embeddings, train_prices, test_prices, train_km, test_km = train_test_split(
    data, embeddings, train['price'], train['km'], test_size=0.2, random_state=42, stratify=train['fuelType'])

# Normalize embeddings
train_embeddings = tf.keras.utils.normalize(train_embeddings)
test_embeddings = tf.keras.utils.normalize(test_embeddings)
train_embeddings = np.transpose(train_embeddings)
test_embeddings = np.transpose(test_embeddings)

In [ ]:
print(test_prices.shape)
print(train['km'].shape)
print(train_km.shape)
print(test_km.shape)

In [ ]:
def process_embedding(embed):
    standard_length = 200
    flat_embed = np.concatenate([np.array(e).flatten() for e in embed])  # Flatten
    # Pad with zeros or truncate to the standard length
    if len(flat_embed) > standard_length:
        return flat_embed[:standard_length]
    elif len(flat_embed) < standard_length:
        return np.pad(flat_embed, (0, standard_length - len(flat_embed)), mode='constant')
    return flat_embed

In [ ]:
train_embeddings = np.array([process_embedding(e) for e in train_embeddings], dtype=np.float32)
train_embeddings = tf.convert_to_tensor(train_embeddings, dtype=tf.float32)

In [ ]:
test_embeddings = np.array([process_embedding(e) for e in test_embeddings], dtype=np.float32)
test_embeddings = tf.convert_to_tensor(test_embeddings, dtype=tf.float32)

In [ ]:
train_data = tf.convert_to_tensor(train_data, dtype=tf.int32)
test_data = tf.convert_to_tensor(test_data, dtype=tf.int32)

In [ ]:
train_prices = tf.convert_to_tensor(train_prices, dtype=tf.float32)
test_prices = tf.convert_to_tensor(test_prices, dtype=tf.float32)

In [ ]:
train_km = tf.convert_to_tensor(train_km, dtype=tf.float32)
train_km = tf.convert_to_tensor(train_km, dtype=tf.float32)

In [ ]:
train_km.shape

In [ ]:
# Text input branch
text_input = Input(shape=(200,), dtype='int32')
text_embed_layer = Embedding(input_dim=verb_size, output_dim=verb_size, input_length=train_data.shape[1])
text_embed = text_embed_layer(text_input)
lstm_out = LSTM(128)(text_embed)

In [ ]:
# Embedding input branch
embed_input = Input(shape=(train_embeddings.shape[1],), dtype = 'int32')
dense_embed = Dense(128, activation='relu')(embed_input)

# KM branch
input_km = Input(shape=(1,), name='km_input')
km_processed = Dense(32, activation='relu')(input_km)

# Concatenate both branches
concat = Concatenate()([lstm_out, dense_embed, km_processed])
hidden = Dense(256, activation='relu')(concat)
output = Dense(1, activation='linear')(hidden)  # Output layer for price prediction

model = Model(inputs=[text_input, embed_input, input_km], outputs=output)
model.summary()

In [ ]:
print(train_data.shape)      # Should be (number of samples, sequence length)
print(train_embeddings.shape)
print(train_km.shape)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_absolute_error'])

In [ ]:
# Train the model
print("COMIENZA EL ENTRENAMIENTO...")
history = model.fit([train_data, train_embeddings, train_km], train_prices,
          validation_data=([test_data, test_embeddings, test_km], test_prices),
          epochs=1, batch_size=64)
print("TERMINA EL ENTRENAMIENTO...")

In [ ]:
plt.xlabel("# Epoca")
plt.ylabel("Loss magnitude")
plt.plot(history.history["loss"])

In [ ]:
test_loss, test_mse = model.evaluate([test_data, test_embeddings, test_km], test_prices)
print(f'Test MAE: {test_mse}')


In [ ]:
predicted_prices = model.predict([test_data, test_embeddings, test_km])

# Assuming test_prices is a 1D array, we can convert it to a DataFrame
results_df = pd.DataFrame({
    'Actual Price': test_prices,
    'Predicted Price': predicted_prices.flatten(),  # Flatten in case it's a 2D array
    'km' : test_km
})

# Display the first few rows of the DataFrame
print(results_df.head())